### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.decomposition import TruncatedSVD

## 1. High Dimensional Data

In [2]:
## loading the data
from sklearn.datasets import fetch_20newsgroups_vectorized

In [3]:
#reading the data
data = fetch_20newsgroups_vectorized()
X = data.data
y = data.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

#### Logistic Regression OnevsRest Algorithm

In [5]:
log_reg_ovr = OneVsRestClassifier(LogisticRegression()).fit(X_train,y_train)

In [6]:
#accuracy on test set
print('The accuracy of Logistic Regression One vs Rest Algorithm is',accuracy_score(y_test,log_reg_ovr.predict(X_test)))

The accuracy of Logistic Regression One vs Rest Algorithm is 0.7998232434821034


#### Logistic Regression OnevsOne Algorithm

In [7]:
log_reg_ovo = OneVsOneClassifier(LogisticRegression()).fit(X_train,y_train)

In [8]:
#accuracy on test set
print('The accuracy of Logistic Regression One vs One Algorithm is',accuracy_score(y_test,log_reg_ovo.predict(X_test)))

The accuracy of Logistic Regression One vs One Algorithm is 0.7220503756076005


#### LDA OnevsRest Algorithm

In [7]:
#To restrict getting out of memory error reduce the dimensions of our data by SVD

In [8]:
svd = TruncatedSVD(n_components=700)
X_train_reduced = svd.fit_transform(X_train)
X_test_reduced = svd.transform(X_test)

In [10]:
lda_ovr = OneVsRestClassifier(LinearDiscriminantAnalysis()).fit(X_train_reduced,y_train)

In [11]:
#accuracy on test set
print('The accuracy of LDA One vs Rest Algorithm is',accuracy_score(y_test,lda_ovr.predict(X_test_reduced)))

The accuracy of LDA One vs Rest Algorithm is 0.821917808219178


#### LDA OnevsOne Algorithm

In [12]:
lda_ovo = OneVsOneClassifier(LinearDiscriminantAnalysis()).fit(X_train_reduced,y_train)

In [13]:
#accuracy on test set
print('The accuracy of LDA One vs Rest Algorithm is',accuracy_score(y_test,lda_ovo.predict(X_test_reduced)))

The accuracy of LDA One vs Rest Algorithm is 0.6827220503756076


## 2. Variable Selection via Forward and Backward Search

In [14]:
from sklearn.linear_model import Lars

Making the Function for Forward and Backward Search

In [33]:
def forward_search(x,y_train,x_valid,y_valid, model,show = True):
    good_features = set() #set to store the values of the features that are good for the model
    good_list = [] #list for convenient indexing
    loss_with_good_features = [] #list to store loss after adding each variable
    n = -1

    all_cols = set(np.arange(x.shape[1]))

    while n<x.shape[1]-1:
        error_dict = {}
        error_dict_train = {}
        n += 1
        active_cols = all_cols.difference(good_features)
        #iterate over all features and add them one by one
        for col in active_cols:

            cols = list(good_features.union(set({col})))
            fx = x[:,cols] #i-th feature with good features

            #fit the model
            if n == 0:
                model.fit(fx.reshape(-1,1),y_train)
            else:
                model.fit(fx,y_train)

            #test it
            preds = model.predict(x_valid[:,cols])
            err = mean_squared_error(y_valid,preds)

            #append the error after adding all the columns
            error_dict[col] = err
            error_dict_train[col] = mean_squared_error(y_train,model.predict(x[:,cols]))

        #get the column which has the lowest error
        good_feature = min(error_dict, key=error_dict.get)

        #store the corresponding value
        loss_with_good_features.append(error_dict[good_feature])

        #make history of good features
        good_features.add(good_feature)
        good_list.append(good_feature)
        
        if show:
            print('Currently Selected Variables',good_features)
            print('The Training Loss is',error_dict_train[good_feature],'The Testing Loss is',error_dict[good_feature],'\n')
        
    return good_list[:np.argmin(loss_with_good_features)+1]

In [34]:
def backward_search(x,y_train,x_valid,y_valid,model,show=True):
    bad_features = set() #set to store the values of the features that are good for the model
    bad_list = [] #list for convenient indexing
    loss_without_bad_features = [] #list to store loss after adding each variable
    n = 0
    all_cols = set(np.arange(x.shape[1]))

    while n<x.shape[1]-1:
        error_dict = {}
        error_dict_train = {}
        n += 1
        active_cols = all_cols.difference(bad_features)
        #iterate over all features and add them one by one
        for col in active_cols:

            cols = list(active_cols.difference(set({col})))
            fx = x[:,cols] #without i-th feature

            #fit the model
            if n == x.shape[1]:
                model.fit(fx.reshape(-1,1),y_train)
            else:
                model.fit(fx,y_train)

            #test it
            preds = model.predict(x_valid[:,cols])
            err = mean_squared_error(y_valid,preds)

            #append the error after adding all the columns
            error_dict[col] = err
            error_dict_train[col] = mean_squared_error(y_train,model.predict(x[:,cols]))

        #get the column which has the lowest error
        bad_feature = min(error_dict, key=error_dict.get)

        #store the corresponding value
        loss_without_bad_features.append(error_dict[bad_feature])

        #make history of good features
        bad_features.add(bad_feature)
        bad_list.append(bad_feature)

        if show:
            print('Currently Selected Variables',active_cols.difference(bad_features))
            print('The Training Loss is',error_dict_train[bad_feature],'The Testing Loss is',error_dict[bad_feature],'\n')
    features = np.delete(list(all_cols),bad_list[:np.argmin(loss_without_bad_features)+1])    
    return features

In [35]:
# reading the data
with open('regression.npy', 'rb') as f:
    X = np.load(f)
    y = np.load(f)

In [36]:
#splitting into trianing and test set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [37]:
#initialize Least Angle Regression Model
least_angle_regression = Lars(normalize=False)

In [38]:
bf = forward_search(X_train,y_train,X_test,y_test,least_angle_regression,False) ##unsorted list of features based on importance
best_features_via_forward_search = sorted(bf)
best_features_via_backward_search = backward_search(X_train,y_train,X_test,y_test,least_angle_regression,False)

In [39]:
#to see if the result of forward and backward search is same
np.all(best_features_via_backward_search == best_features_via_forward_search)

True

In [40]:
##to see what are those features
print('The indices of the selected features are')
print(best_features_via_backward_search,'\n')

##to fit a model using those features
least_angle_regression = Lars(normalize=False)
least_angle_regression.fit(X_train[:,best_features_via_backward_search],y_train)
##the error produced by the features selected through feature selection
print('The error produced by the feature selection model LARS is:',mean_squared_error(y_test,least_angle_regression.predict(X_test[:,best_features_via_backward_search])))

The indices of the selected features are
[ 0  1  3  4  8  9 11 12 13 17 19 24 27 32 33 35 37 38 40 48 51 55 58 59
 60 61 62 64 65 68 69 71 83 84 90 92 93 94 95] 

The error produced by the feature selection model LARS is: 5.994330195060494


## 3. Regularization

**Small and Large Value of Alpha:**
If the value of alpha is very high, the regularization term will overpower the loss function and the model would just learn to optimize on the regularization term. This would not result in any fruitful model training (chances of underfitting). On the other hand if the value of alpha is very low, the model will ignore the regularization and instead weigh more the optimization of the cost function itself, which would be close to having no regularization as in simple linear regression.

In [41]:
from sklearn.linear_model import Ridge,ElasticNet,Lasso
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [42]:
#initialize ridge, lasso and elastic net model
ridge = Ridge()
elastic_net = ElasticNet()
lasso = Lasso()
clf_list = [ridge,elastic_net,lasso]

In [43]:
#param grid with different values of alpha
params = {'alpha':[10,1,0.1,0.0001,0.00001]}

In [44]:
best_models = []
best_errors = []
best_estimators = []
for clf in clf_list:
    #run grid search
    grid_search = GridSearchCV(clf,param_grid=params,scoring='neg_mean_squared_error',cv=5,refit=True)
    grid_search.fit(X_train,y_train)
    best_models.append(grid_search)
    best_estimators.append(np.array(grid_search.best_estimator_))
    best_errors.append(grid_search.best_score_)
    
    #run random search
    random_search = RandomizedSearchCV(clf,params,scoring='neg_mean_squared_error',cv=5,refit=True,n_iter=3)
    random_search.fit(X_train,y_train)
    best_models.append(random_search)
    best_estimators.append(np.array(random_search.best_estimator_))
    best_errors.append(random_search.best_score_)

In [45]:
results = pd.DataFrame(best_models,columns=['models'])
results['errors'] = best_errors

In [46]:
results['estimators'] = results['models'].apply(lambda x: np.array(x.estimator))
results['alpha'] = results['models'].apply(lambda x:x.best_params_['alpha'])


In [47]:
results.sort_values(by='errors',ascending=False)

,models,errors,estimators,alpha
4,"GridSearchCV(cv=5, estimator=Lasso(),\n ...",-5.963695,Lasso(),0.00010
5,"RandomizedSearchCV(cv=5, estimator=Lasso(), n_...",-5.963695,Lasso(),0.00010
2,"GridSearchCV(cv=5, estimator=ElasticNet(),\n ...",-5.964449,ElasticNet(),0.00010
0,"GridSearchCV(cv=5, estimator=Ridge(),\n ...",-5.965118,Ridge(),1.00000
1,"RandomizedSearchCV(cv=5, estimator=Ridge(), n_...",-5.965118,Ridge(),1.00000
3,"RandomizedSearchCV(cv=5, estimator=ElasticNet(...",-5.965309,ElasticNet(),0.00001


**Top 3 Models:** here we can see that the Lasso with 0.0001 alpha value, ElasticNet with 0.0001 alpha value and Ridge with alpha value of 1 are top three models

**Comparision with Q2:** Also, One more thing we can observe is these models are slightly better than the one we obtained through backward and forward search. That model gave MSE = 5.99 however the top 3 regularized models give error of 5.96

Let's obtain the Top K indices of these three models and compare them with the ones obtained through forward search and LARS

In [50]:
r = Ridge(alpha=1)
l = Lasso(alpha=0.0001)
e = ElasticNet(alpha=0.0001)
best_three_models = [r,l,e]

In [51]:
indices = []
for m in best_three_models:
    indices.append(forward_search(X_train,y_train,X_test,y_test,m,False)) 

Let's see if the top K indices of best three models are same as LARS model in Q2

In [69]:
## trying out bunch of K return True if the the value of all three models is same as Q2 model
for k in range(1,11):
    print('The current K is',k)
    same = []
    tk_l = bf[:k]
    ## compairing with top K indices of LARS model
    for ind in indices:
    same.append(np.all(ind[:k] == tk_l))
    print(np.all(same))

The current K is 1
True
The current K is 2
True
The current K is 3
True
The current K is 4
True
The current K is 5
True
The current K is 6
True
The current K is 7
True
The current K is 8
False
The current K is 9
False
The current K is 10
False


Upto K=7 the indices are same

## 4. Custom Linear Regression Using Inheritance

In [2]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.estimator_checks import check_estimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [3]:
class MyLinearRegression(BaseEstimator,RegressorMixin):
    
    def fit(self, X, y):
        #check if the shape of X and y is correct
        X, y = check_X_y(X, y)
        
        x_bar = np.mean(X,axis=0) # calculate mean of x
        y_bar = np.mean(y,axis=0) # calculate mean of y

        # for calculating B1
        B1 = np.divide(np.sum(np.multiply((X-x_bar),(y-y_bar).reshape(-1,1)),axis=0,keepdims=True),np.sum(np.square(X-x_bar),axis=0,keepdims=True))

        # for calculating B0
        B0 = y_bar - B1.reshape(1,-1)@x_bar.reshape(-1,1)
        
        self.coef_ = B1.T
        self.intercept_ = B0
        self.n_features_in_ = X.shape[1]
        
        return self
    
    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        return np.ravel(X@self.coef_ + self.intercept_)

In [4]:
#making the data
X = np.arange(0,5.5,0.5)
y = np.array([6.0,4.83,3.7,3.15,2.41,1.83,1.49,1.21,0.96,0.73,0.64])

#splitting into trianing and test set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
#fitting the model
linear_reg = MyLinearRegression()
linear_reg.fit(X_train.reshape(-1,1),y_train) ## reshaping to avoid the error

MyLinearRegression()

In [6]:
## to check if the estimator is valid
check_estimator(MyLinearRegression())

In [7]:
##predicting on test set
preds = linear_reg.predict(X_test.reshape(-1,1))
mean_squared_error(y_test,preds)

0.7237418923740918

In [8]:
##print the value of B0 and B1
print('The value of B0 is',linear_reg.intercept_)
print('The value of B1 is',linear_reg.coef_)

The value of B0 is [[4.63811252]]
The value of B1 is [[-0.91292196]]
